In [6]:
import datetime


import warnings

import pandas as pd
from binance.spot import Spot
from MultiTrade.live_environments import LiveTradingEnv
from neuralforecast.core import NeuralForecast
from utils.utils import build_market_image, sharpe_reward

warnings.filterwarnings('ignore')


In [5]:
image=build_market_image(target_pair='ETH/USDT',time_frame='1h',axis=0)

In [ ]:
datetime. datetime.now()-pd.Timedelta('1h').to_pytimedelta()

In [ ]:
trade_target='BTC/USDT'
api_key='ZGvDtoVQM9O1xb0PXaSPzMffWtGH3ofxVSbsuzGQNDBwklCHkKm4x1g3nrImqQXF'
api_secret='BUDui1xmoP11wGmSV3RqTrtrkmYU5EyogHMtigVQgTGoQxjcibkuwjN5Et169f4o'

client =Spot(api_key=api_key,api_secret=api_secret,base_url='https://testnet.binance.vision')
client


In [ ]:
# account=client.account()

In [ ]:
# client.avg_price('ETHUSDT'),client.ticker_price('ETHUSDT')


In [ ]:

model=NeuralForecast.load("C:/Users/standard/Git/MultiTrader/forecaster_pretrain/neuralForecast")

In [ ]:
live_env_config=dict(
                name='ETHUSDT_train',
                model=model,
                api_key=api_key,
                api_secret=api_secret,
                test_net=True,
                restore_trading=False,
                target_symbol='ETH/USDT',
                time_frame='1h',
                reward_function=sharpe_reward,
                positions = [ -.5,-.25,.25, .5], # -1 (=SHORT), 0(=OUT), +1 (=LONG)
                trading_fees = 0.01/100, # 0.01% per stock buy / sell (Binance fees)
                borrow_interest_rate= 0.0003/100, # 0.0003% per timestep (one timestep = 1h here)
                max_episode_duration=168,
                verbose=0,
                )

In [ ]:
self=LiveTradingEnv(**live_env_config)
self.df

In [ ]:
obs,info=self.reset(reset_account=False)
obs.shape


In [ ]:
from ray.rllib.policy.policy import Policy

In [ ]:
agent= Policy.from_checkpoint('C:/Users/standard/Git/MultiTrader/Rayrl_tune/checkpoints/policies/default_policy')

agent.export_model('C:/Users/standard/Git/MultiTrader/Rayrl_tune/checkpoints/pretrained_agent')


In [ ]:
# action,_,states=agent.compute_single_action(obs)
# obs, reward, terminated, truncated, info=self.live_step(action,wait=False)
# action,_,states=agent.compute_single_action(obs)


In [ ]:
obs,info=self.reset(reset_account=True)
while True:
    action,_,states=agent.compute_single_action(obs)
    obs, reward, terminated, truncated, info=self.live_step(action,wait=True)

In [ ]:
self.df.iloc[-1]

In [ ]:
# h=self.historical_info[0]
history_df=pd.DataFrame([self.historical_info[-1]])
history_df

In [ ]:
import sqlite3 as db




# connect to a test DB with one three-column table titled "Demo"
conn = db.connect('Trade_history/trade.db')
cur = conn .cursor()

In [ ]:
history_df.to_sql(f'{self.symbol}_trade_history',conn,if_exists='append')

In [ ]:
history_df=pd.read_sql(f'select * from {self.symbol}_trade_history',conn)
history_df

In [ ]:
self.historical_info[-1]

In [ ]:
for h in self.historical_info:
    insert_sql = f'INSERT INTO {self.symbol}_trade_history {h}' 
    conn.execute(insert_sql)

In [ ]:
client.klines("BNBUSDT", "1h", limit=1)

In [ ]:
self._get_ticker()

In [ ]:
data_list=client.klines("BNBUSDT", "1h", limit=10)
columns=['date_open','open','high','low','close','volume','date_close','QA_volume','N_trades','BA_volume','BQ_volume','unused']
data=pd.DataFrame(data_list,columns=columns)
data['date_close']=data['date_close'].apply(pd.to_datetime,unit='ms')
data['date_open']=data['date_open'].apply(pd.to_datetime,unit='ms')

# data_list

In [ ]:
data